In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import ttest_ind, f_oneway 
from feature_engine.encoding import RareLabelEncoder
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [4]:
data.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [5]:
data.isna().sum()
data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [6]:
main_label = 'price'
# Exclude 1% of smallest and 1% of highest prices
P = np.percentile(data[main_label], [1, 99])
df = data[(data[main_label] > P[0]) & (data[main_label] < P[1])]
# combine neighbourbood and neighbourhood_group
df['neighbourhood'] = df['neighbourhood'] + ', ' + df['neighbourhood_group']
# log10-transform columns and group for larger bins
for col in ['minimum_nights', 'number_of_reviews', 'calculated_host_listings_count', 'availability_365']:
    df[f'log10_{col}'] = df[col].apply(lambda x: 1/5*round(5*np.log10(1+x)))
    df = df.drop([col], axis=1)
# set up the rare label encoder limiting number of categories to max_n_categories
for col in ['neighbourhood', 'room_type']:
    encoder = RareLabelEncoder(n_categories=1, max_n_categories=70, replace_with='Other', tol=20/df.shape[0])
    df[col] = encoder.fit_transform(df[[col]])
# drop unused columns
cols2drop = ['name', 'host_id', 'host_name', 'latitude', 'longitude', 'neighbourhood_group', 
             'last_review', 'reviews_per_month']
df = df.drop(cols2drop, axis=1)
print(df.shape)
df.sample(5).T

<ipython-input-6-577a6b75d478>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neighbourhood'] = df['neighbourhood'] + ', ' + df['neighbourhood_group']
<ipython-input-6-577a6b75d478>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'log10_{col}'] = df[col].apply(lambda x: 1/5*round(5*np.log10(1+x)))


(47744, 8)


,38980,6064,37644,35908,20231
id,30464086,4438470,29852402,28519623,16179862
neighbourhood,"East Flatbush, Brooklyn","East Village, Manhattan","Bushwick, Brooklyn","Williamsburg, Brooklyn","Bedford-Stuyvesant, Brooklyn"
room_type,Entire home/apt,Private room,Shared room,Private room,Private room
price,150,105,42,105,75
log10_minimum_nights,0.4,0.4,0.4,0.4,0.4
log10_number_of_reviews,1.2,1.0,0.8,1.8,0.0
log10_calculated_host_listings_count,0.4,0.4,0.4,0.4,0.4
log10_availability_365,2.2,2.2,0.0,1.8,0.0


In [7]:
X = df[['neighbourhood','room_type', 'log10_minimum_nights','log10_number_of_reviews','log10_calculated_host_listings_count','log10_availability_365']]
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,neighbourhood,room_type,log10_minimum_nights,log10_number_of_reviews,log10_calculated_host_listings_count,log10_availability_365
1986,"Flatbush, Brooklyn",Entire home/apt,0.4,1.4,0.4,2.4
28894,"Astoria, Queens",Entire home/apt,2.0,0.0,0.4,0.0
32731,"Upper West Side, Manhattan",Private room,0.4,1.8,0.6,1.6
28458,"Williamsburg, Brooklyn",Private room,0.4,0.4,0.4,0.0
33156,"Crown Heights, Brooklyn",Entire home/apt,0.4,1.4,0.4,2.6
...,...,...,...,...,...,...
11473,"Lower East Side, Manhattan",Entire home/apt,0.6,0.8,0.4,0.0
45778,"Upper West Side, Manhattan",Entire home/apt,0.6,0.6,0.4,2.4
39002,"Bushwick, Brooklyn",Entire home/apt,0.4,1.0,0.4,0.8
867,"Harlem, Manhattan",Private room,0.6,1.6,0.6,2.4


In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), [ 'neighbourhood', 'room_type'])
    ])

# Pipeline
baseline_model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

baseline_model.fit(X_train, y_train)
y_pred = baseline_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print('baseline_model RMSE:', rmse)
r2 = r2_score(y_test,y_pred)
print("R-squared:", r2)

baseline_model RMSE: 80.03164894472849
R-squared: 0.3719938873838401


In [9]:
# decision_tree baseline model

# Define the preprocessor as part of the pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), [ 'neighbourhood', 'room_type'])
    ])

# Pipeline with decision tree regressor
tree_model = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', DecisionTreeRegressor())])

# Parameter grid for GridSearchCV
param_grid = {
    'regressor__max_depth': [2, 5, 10, 20],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

# Grid search with cross-validation
grid_search = GridSearchCV(tree_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best estimator found by grid search
print('Best parameters found:', grid_search.best_params_)

# Predict using the best model
y_pred = grid_search.predict(X_test)

# Calculate RMSE and R-squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print('Decision Tree RMSE:', rmse)
print('R-squared:', r2)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters found: {'regressor__max_depth': 20, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2}
Decision Tree RMSE: 80.21744084505924
R-squared: 0.36907469518960423


In [10]:
# Define the preprocessor as before
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), [ 'neighbourhood', 'room_type'])
    ])

# Define the pipeline with a random forest regressor
rf_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor())])

# Define the parameter grid for the RandomForestRegressor
param_grid = {
    'regressor__n_estimators': [100,200,300],  # The number of trees in the forest
    'regressor__max_depth': [3,4,5,6,7,8,10,20],  # The maximum depth of the trees
}

# Create the GridSearchCV object
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best estimator
print('Best parameters found:', grid_search.best_params_)

# Predict using the best model
y_pred = grid_search.predict(X_test)

# Calculate RMSE and R-squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print('Random Forest RMSE:', rmse)
print('R-squared:', r2)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters found: {'regressor__max_depth': 20, 'regressor__n_estimators': 200}
Random Forest RMSE: 80.15992600492658
R-squared: 0.3699791009848383


In [11]:
# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), [ 'neighbourhood', 'room_type'])
    ])

# Pipeline with XGBoost regressor
xgb_model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', XGBRegressor())])

# Parameter grid for XGBoost
param_grid = {
    'regressor__n_estimators': [100, 200],  # Number of gradient boosted trees. Equivalent to number of boosting rounds
    'regressor__learning_rate': [0.01, 0.1],  # Boosting learning rate (xgb's "eta")
    'regressor__max_depth': [3, 6],  # Maximum depth of a tree
    'regressor__subsample': [0.7, 1],  # Subsample ratio of the training instances
    'regressor__colsample_bytree': [0.7, 1]  # Subsample ratio of columns when constructing each tree
}

# Create the GridSearchCV object
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best estimator found by grid search
print('Best parameters found:', grid_search.best_params_)

# Predict using the best model
y_pred = grid_search.predict(X_test)

# Calculate RMSE and R-squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print('XGBoost Regressor RMSE:', rmse)
print('R-squared:', r2)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters found: {'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 6, 'regressor__n_estimators': 200, 'regressor__subsample': 0.7}
XGBoost Regressor RMSE: 79.48815289981526
R-squared: 0.3804945216298158


In [12]:
# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), [ 'neighbourhood', 'room_type'])])

# Define the pipeline with Lasso regression
lasso_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', Lasso())])

# Define the parameter grid to search
param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(lasso_pipeline, param_grid, cv=5)

# Fit the model
grid_search.fit(X_train, y_train)

# Predict and evaluate
y_pred_lasso = grid_search.predict(X_test)

rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)

print('Lasso Model RMSE:', rmse_lasso)
print("Lasso Model R-squared:", r2_lasso)

c:\Users\gjj71\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4690061.387602776, tolerance: 32554.25665096217
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\gjj71\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1966995.4553709924, tolerance: 32983.06456657613
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\gjj71\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2031713.936957121, tolerance: 32377.185285518397
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\gjj71\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py

Lasso Model RMSE: 80.03124857802906
Lasso Model R-squared: 0.3720001707007262


c:\Users\gjj71\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2512319.3087700903, tolerance: 40803.097361104854
  model = cd_fast.sparse_enet_coordinate_descent(


In [13]:
# Define the preprocessor as before
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), [ 'neighbourhood', 'room_type'])
    ])

# Pipeline with SVM regressor
svm_model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', SVR())])

# Parameter grid for SVR
param_grid = {
    'regressor__C': [0.1, 1, 10],  # Regularization parameter
    'regressor__epsilon': [0.01, 0.1, 0.5],  # Epsilon in the epsilon-SVR model
    'regressor__kernel': ['linear', 'poly', 'rbf', 'sigmoid']  # Type of kernel function
}

# Create the GridSearchCV object
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best estimator found by grid search
print('Best parameters found:', grid_search.best_params_)

# Predict using the best model
y_pred = grid_search.predict(X_test)

# Calculate RMSE and R-squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print('SVM Regressor RMSE:', rmse)
print('R-squared:', r2)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters found: {'regressor__C': 10, 'regressor__epsilon': 0.01, 'regressor__kernel': 'poly'}
SVM Regressor RMSE: 82.67214247847497
R-squared: 0.32987051606872253


In [14]:
# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), ['neighbourhood', 'room_type'])
    ])

# Define the pipeline with Gradient Boosting Regressor
gbr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Define the parameter grid to search
param_grid = {
    'regressor__n_estimators': [100, 200],  # Number of boosting stages to perform
    'regressor__learning_rate': [0.01, 0.1],  # Learning rate shrinks the contribution of each tree
    'regressor__max_depth': [3, 4, 5],  # Maximum depth of the individual regression estimators
    'regressor__min_samples_split': [2, 3],  # The minimum number of samples required to split an internal node
    'regressor__min_samples_leaf': [1, 2]  # The minimum number of samples required to be at a leaf node
}

# Create the GridSearchCV object
grid_search = GridSearchCV(gbr_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Best estimator found by grid search
print('Best parameters found:', grid_search.best_params_)

# Predict and evaluate
y_pred_gbr = grid_search.predict(X_test)

# Calculate RMSE and R-squared
rmse_gbr = mean_squared_error(y_test, y_pred_gbr, squared=False)
r2_gbr = r2_score(y_test, y_pred_gbr)

print('Gradient Boosting Regressor RMSE:', rmse_gbr)
print('Gradient Boosting Regressor R-squared:', r2_gbr)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters found: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 5, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
Gradient Boosting Regressor RMSE: 79.56817493704085
Gradient Boosting Regressor R-squared: 0.3792465609672452


In [15]:
# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OneHotEncoder(), ['neighbourhood', 'room_type'])
    ])

# Define the pipeline with Elastic Net regression
elastic_net_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ElasticNet(random_state=42))
])

# Define the parameter grid to search
param_grid = {
    'regressor__alpha': [0.1, 1, 10],  # Constant that multiplies the penalty terms
    'regressor__l1_ratio': [0.1, 0.5, 0.9],  # The ElasticNet mixing parameter
    'regressor__max_iter': [1000]  # Number of iterations to converge
}

# Create the GridSearchCV object
grid_search = GridSearchCV(elastic_net_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Best estimator found by grid search
print('Best parameters found:', grid_search.best_params_)

# Predict and evaluate
y_pred_elastic_net = grid_search.predict(X_test)

# Calculate RMSE and R-squared
rmse_elastic_net = mean_squared_error(y_test, y_pred_elastic_net, squared=False)
r2_elastic_net = r2_score(y_test, y_pred_elastic_net)

print('Elastic Net RMSE:', rmse_elastic_net)
print('Elastic Net R-squared:', r2_elastic_net)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters found: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.9, 'regressor__max_iter': 1000}
Elastic Net RMSE: 81.74481561191755
Elastic Net R-squared: 0.34481977940387276


In [16]:
# initialize data
y = df[main_label].values.reshape(-1,)
X = df.drop([main_label], axis=1)
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols_idx = [list(X.columns).index(c) for c in cat_cols]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# initialize Pool
train_pool = Pool(X_train, 
                  y_train, 
                  cat_features=cat_cols_idx)
test_pool = Pool(X_test,
                 y_test,
                 cat_features=cat_cols_idx)
# specify the training parameters 
model = CatBoostRegressor(iterations=1000, 
                          depth=5,
                          verbose=0,
                          learning_rate=0.01, 
                          loss_function='RMSE')
# train the model
model.fit(train_pool)

# Predict using the best model
y_pred = model.predict(X_test)

# Calculate RMSE and R-squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print('catboost Regressor RMSE:', rmse)
print('R-squared:', r2)

catboost Regressor RMSE: 74.12583484223778
R-squared: 0.4612596102807398
